In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import YoutubeLoader, UnstructuredURLLoader
import traceback
import validators
import openai

In [8]:

def is_valid_url(url=''):
    return validators.url(url)

def get_content(url):
    loader = ''
    if "youtube.com" in url.lower():
        loader = YoutubeLoader.from_youtube_url(youtube_url=url, add_video_info=True, language=["en", "id"],translation="en")
    else:
        loader=UnstructuredURLLoader(urls=[url],ssl_verify=False,
                headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"})
    return loader

def get_summarized_content(loader, model, prompt):
    output_summary = ''
    if(loader):
        docs=loader.load() if loader else {}
        # chain = load_summarize_chain(model, chain_type="stuff", prompt=prompt, document_variable_name="page_content" )
        chain = load_summarize_chain(model, chain_type="stuff", prompt=prompt, document_variable_name="page_content" )
        print(docs, "docs", chain)
        output_summary = chain.invoke(docs)
        print(output_summary, "output_summary")
    return output_summary

In [9]:
url = 'https://learningsciences.smu.edu/blog/artificial-intelligence-in-education'

In [10]:
import os
model_name = os.getenv("MODEL_NAME", "gpt-3.5-turbo")
print(model_name, "model_name")
try:    
        prompt_template="""
            Provide a summary of the following content in 300 words:
            Content:{page_content}

            """
        prompt=PromptTemplate(template=prompt_template,input_variables=["page_content"])


        if(is_valid_url(url)):
            loader = get_content(url)
            # llm = connect_to_model(model_name)
            api_key = os.getenv("OPEN_AI_API_KEY")
            TEMP = os.getenv("TEMPERATURE", 0.2)
            llm = ChatOpenAI(openai_api_key=api_key, model=model_name, temperature=TEMP)
            # print(llm, "llm")
            summarized_content = get_summarized_content(loader, llm, prompt)
            print(summarized_content, "summarized_content")
            # summarized_content = llm.invoke("Hello,")
            if hasattr(summarized_content, "input_documents"):
                print("true")
            summarized_content = summarized_content.input_documents[0].page_content
            print(summarized_content, "summarized_content")

            # return {"status": "ok", "summary": summarized_content}
            # return {"status": "ok", "summary": summarized_content}
        # else:
            # return {"status": "Failed", "message": "Invalid URL"}
except Exception as e:
    traceback.print_exc()
    # return {"status": "Failed", "message": str(e)}

gpt-5-mini-2025-08-07 model_name


/Users/kalyanjyothula/anaconda3/envs/flask-api-env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'learningsciences.smu.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'source': 'https://learningsciences.smu.edu/blog/artificial-intelligence-in-education'}, page_content="Artificial Intelligence (AI) is swiftly changing the educational landscape, bringing about significant benefits and noteworthy challenges. In classrooms across the globe, AI's influence can be seen through advanced technologies like machine learning, which open up new possibilities for personalized learning and enhanced student engagement.\n\nThis blog aims to unpack the concept of AI in education, breaking down its key components, current applications, and the very real benefits it brings to learning environments. We will also address common concerns, such as the balance between technology and traditional teaching, and the implications for privacy and standardization.\n\nMoreover, we'll provide practical insights for educators on integrating AI into their teaching practices and discuss the importance of training, such as through advanced degrees like an online M.S

KeyboardInterrupt: 

In [12]:
summarized_content

{'input_documents': [Document(metadata={'source': 'https://learningsciences.smu.edu/blog/artificial-intelligence-in-education'}, page_content="Artificial Intelligence (AI) is swiftly changing the educational landscape, bringing about significant benefits and noteworthy challenges. In classrooms across the globe, AI's influence can be seen through advanced technologies like machine learning, which open up new possibilities for personalized learning and enhanced student engagement.\n\nThis blog aims to unpack the concept of AI in education, breaking down its key components, current applications, and the very real benefits it brings to learning environments. We will also address common concerns, such as the balance between technology and traditional teaching, and the implications for privacy and standardization.\n\nMoreover, we'll provide practical insights for educators on integrating AI into their teaching practices and discuss the importance of training, such as through advanced degree

In [ ]:
summary = summarized_content['output_text']



In [16]:
len(summary)

2438

In [20]:
type(summarized_content)
summarized_content.keys()
'output_text' in summarized_content

True

In [21]:
input_doc = summarized_content['input_documents'][0].page_content

In [23]:
len(input_doc)
len(summary)

2438

In [11]:
import validators
import requests
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
# from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains.summarize import load_summarize_chain
from transformers import pipeline, AutoProcessor, AutoModelForSpeechSeq2Seq
from langchain_community.document_loaders import YoutubeLoader, UnstructuredURLLoader
from pytubefix import YouTube
from moviepy import AudioFileClip
from langchain_core.documents import Document
import openai


In [12]:
def connect_to_model(model_name):
    api_key = os.getenv("OPEN_AI_API_KEY")
    TEMP = os.getenv("TEMPERATURE", 0.2)
    llm = ChatOpenAI(openai_api_key=api_key, model=model_name, temperature=TEMP)
    return llm


In [ ]:
# openai.api_key = os.getenv("OPENAI_API_KEY")
# os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

TypeError: str expected, not NoneType

In [36]:
import openai
def audio_to_text_content(url):
    try:
        transcription = ''
        file_path = ''
        print(url, "url at start")
        if "youtube.com" in url.lower():
            yt = YouTube(url)
            video = yt.streams.get_audio_only()
            downloaded_file = video.download()
            # print(downloaded_file, "downloaded_file")
            # Convert the audio file to a .wav format
            
            audio_clip = AudioFileClip(downloaded_file)
            new_path = downloaded_file.replace(' ','_')
            file_path = os.path.splitext(new_path)[0] + '.wav'
            # print(file_path, "file_path")
            audio_clip.write_audiofile(file_path)
            transcription = transcribe_audio_file(file_path)
            os.remove(downloaded_file)
        else:
            # print("else")
            file_path = download_audio_file(url, "temp_audio_file")
            transcription = transcribe_audio_file(file_path)

        # Clean up files
        if os.path.exists(file_path): os.remove(file_path)
        return transcription
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
 
def transcribe_audio_file(file_path):
    api_key = os.getenv("OPEN_AI_API_KEY")
    os.environ['OPENAI_API_KEY'] = api_key
    transcript = ''
    with open(file_path, "rb") as audio_file:
        print("inside with")
        transcript = openai.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file,
                    # api_key=api_key
                )
        print(transcript, "transcript")
    return transcript
    
    AUDIO_MODEL = os.getenv("AUDIO_MODEL")
    print(AUDIO_MODEL)
    processor = AutoProcessor.from_pretrained(AUDIO_MODEL)
    audio_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL)
    # print(audio_model, "audio_model")
    audio_pipe = pipeline(
        "automatic-speech-recognition",
        model=audio_model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        # device=-1  # Use CPU
    )
    # print(audio_pipe, "audio_pipe")
    result = audio_pipe(file_path, return_timestamps=True)
    return result['text']
    # audio_input, _ = librosa.load(file_path, sr=16000)
    # input_features = processor(audio_input, return_tensors="pt").input_features
    # with torch.no_grad():
    #         predicted_ids = audio_model.generate(input_features)
    #         transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription
def download_audio_file(audio_url, save_path):
    response = requests.get(audio_url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        return save_path
    else:
        raise Exception("Failed to download audio file")
    

In [29]:
def is_valid_url(url=''):
    return validators.url(url)

def get_summarized_content_audio(loader, model, prompt):
    output_summary = ''
    if(loader):
        # docs = [Document(page_content=loader)]
        chain = load_summarize_chain(model, chain_type="stuff", prompt=prompt, document_variable_name="page_content"  )
        output_summary = chain.invoke(loader)
    return output_summary

In [23]:
audio_data = "Pakistan is back to default. After Hawking Rare Earths at the White House, Islamabad is turning to a tried and tested strategy, blaming India. Because Pakistan's domestic situation is getting worse. There are protests in Pakistan occupied Kashmir. There are terror attacks in Balochistan. The economy is as bad as always. So, on cue, the country's Defense Minister turned to India. He says the risk of war with India is very real. Our next report tells you more. When trouble comes, Pakistan has just one response. Turn to India. And right now, Pakistan is facing a deluge of trouble. In Khyber Pakhtunkwa, there is a political and security crisis. The province is ruled by Imran Khan's PTI party, but its Chief Minister has resigned after being asked to by the party chief. Why? Over-repeated security failures. Just yesterday, the Pakistani Taliban attacked and killed 11 military personnel. It's part of a worrying trend in the province. So far, in 2025, Kaiba Pakhtunkwa has seen 605 terrorist attacks. Overall, militancy and violence in Pakistan has surged by 46% this year, and Kaiba is the worst-hit region. By asking the Chief Minister to resign, Imran Khan is hoping to appear tough. A contrast when you look at the Pakistani army failures, they seem unable to protect their own, let alone civilians. Over in Sindh, it was almost the deja vu moment this week. Militants targeted the Jaffar Express which runs from Quetta to Peshawar. A bomb blast on the railway line derailed four bogies of the train. It left at least seven passengers injured. The same train was targeted by Baloch separatists in March of 2025. They attacked and hijacked the train, holding at least 380 passengers hostage, while most were rescued, around 30 of them were killed. If you move to Pakistan occupied Kashmir, the story is equally grim. Deadly protests rocked the region last week. Those were demanding legislative reforms and cheaper food and electricity. The government responded with a brutal crackdown. It left around 12 protesters dead. All these security challenges have gripped Pakistan in the middle of an economic crisis. The World Bank has slashed the country's growth forecast. the monsoon season, it was expected to grow by 3.1%. But then came the ranks. Pakistan's Punjab province faced its worst flooding in years. As a result, the agriculture sector, the backbone of Pakistan's economy, suffered. The World Bank is now expecting the country to grow by just 2.6 percent. Amid all these crises, you would expect their leadership to focus inward, but you'd be wrong. Pakistan's military and civilian leaders are warmongering. Listen to their Defence Minister, Qawwaja Asif. I do not want escalation, but the risks are real, and I am not denying that. If it comes comes to war, we will achieve a better result than before. Pakistan's military is also saber rattling. Earlier this week, indian military officers had won pakistan against sponsoring terrorists. An innocent party would have denied the charge altogether. But not pakistan. Their military warned india of a swift response against any action. In many ways, this sums up the problem with Pakistan. The country has a hundred problems, but it keeps obsessing over the imagined threat from India. you"

In [38]:
url = 'https://www.audiocheck.net/download.php?filename=Audio/audiocheck.net_hdsweep_1Hz_44000Hz_-3dBFS_30s.wav'
# url="https://file-examples.com/storage/fe2b352ea268ea149938125/2017/11/file_example_WAV_5MG.wav"
model_name = os.getenv("MODEL_NAME")
url='https://www.youtube.com/watch?v=iCoGRriiVug'
 
prompt_template="""
    Provide a summary of the following content in 500 words and please use subtitles and bullet points whenever necessary.
    Make sure the summary is easy to understand and captures the main points effectively. Avoid using overly technical language or jargon, and aim for clarity and conciseness.:
    Content:{page_content}
    """
prompt=PromptTemplate(template=prompt_template,input_variables=["page_content"])

if(is_valid_url(url)):
    # print("valid url start", url)
    loader = audio_to_text_content(url)
    print(loader, "loader")
    # loader = audio_data
    loader = [Document(page_content=loader)]
    
    llm = connect_to_model(model_name)
    summarized_content = get_summarized_content_audio(loader, llm, prompt )
    if 'output_text' in summarized_content:
        summarized_content = summarized_content['output_text']
    print(summarized_content, "summarized_content")

https://www.youtube.com/watch?v=iCoGRriiVug url at start
MoviePy - Writing audio in /Users/kalyanjyothula/Desktop/Home/GenAI/space-app/text_api/Vizag_&_Mumbai_Chosen_for_Meta’s_Mega_Subsea_Cable_Landing.wav


MoviePy - Done.
inside with
An error occurred: Error code: 400 - {'error': {'message': 'something went wrong reading your request', 'type': 'invalid_request_error', 'param': None, 'code': None}}
None loader


ValidationError: 1 validation error for Document
page_content
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type